# <center> Индивидуальное задание по модулю 5
# <center> «Python для автоматизации анализа данных»

Выполнила: **Коломыцева Анна**

## Описание задачи:
**Датасет:** Список лэптопов с параметрами для выбора
https://www.kaggle.com/datasets/rajugc/laptop-selection-dataset

Этот набор данных содержит информацию о 1000 ноутбуках, доступных на индийском веб-сайте электронной коммерции Flipkart. Данные включают технические характеристики, такие как тип процессора, объем оперативной памяти, емкость жесткого диска, размер экрана, а также операционную систему. Кроме того, набор данных включает рейтинги клиентов и отзывы, а также информацию о ценах. 

**Цель:** мы хотим выпустить ноутбук, но для этого нам нужно понимать оптимальные параметры для него, следовательно основываясь на пользовательском рейтинге нам нужно выбрать оптимальную ОС, размер хранилища и т.п.

**Гипотеза:** если мы выпустим ноутбук с наиболее востребованными характеристиками, то он будет лучше продаваться.

## План работы:
1. Проверить качество данных:
    - проверить типы данных колонок и приведите их к нужному типу;
    - проверить наличие пропущенных значений и обработайте их;
    - проверить данные на наличие дубликатов, и, если необходимо, вычистить их.
2. Рассчитать основные статистические показатели для числовых переменных, такие как среднее, медиана и стандартное отклонение. Другие статистические показатели также могут быть рассчитаны в зависимости от конкретных целей и задач анализа данных. Сделать вывод по результатам.
3. Построить графики, наиболее релевантные для анализа данных. Графики должны быть информативными и помочь лучше понять данные. При необходимости ввести дополнительные колонки в исходный датафрейм.
После построения графиков проанализировать полученные результаты и сделать выводы (например, какие данные имеют наибольшую
значимость, какие переменные влияют на результаты). Если это необходимо, доработать с данными на этом этапе.
4. Вычислить матрицу корреляции и построть heatmap для нее. Сделать выводы о взаимосвязях, обнаруженных в данных.
5. Сделать общий вывод по исследуемому датасету. Предложить возможные способы применения полученных результатов анализа данных. 

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
import plotly.express as px #для визуализации
import re # регулярные выражения для преобразования данных

In [2]:
# чтение датасета
data = pd.read_csv('laptops.csv', index_col='Unnamed: 0')
display(data.sample(3))

,img_link,name,price(in Rs.),processor,ram,os,storage,display(in inch),rating,no_of_ratings,no_of_reviews
631,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo V15 G2 Core i3 11th Gen,37500,Intel Core i3 Processor (11th Gen),8 GB DDR4 RAM,64 bit Windows 11 Operating System,1 TB HDD|256 GB SSD,15.6,4.4,53.0,3.0
226,https://rukminim1.flixcart.com/image/312/312/x...,ASUS Vivobook 15 Ryzen 3 Dual Core AMD R3,34990,AMD Ryzen 3 Dual Core Processor,8 GB DDR4 RAM,Windows 11 Operating System,512 GB SSD,15.6,4.4,405.0,48.0
678,https://rukminim1.flixcart.com/image/312/312/x...,MSI Core i7 13th Gen,147990,Intel Core i7 Processor (13th Gen),16 GB DDR5 RAM,Windows 11 Operating System,1 TB SSD,17.3,NaN,NaN,NaN


## 1. Проверка качества данных

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 984 entries, 0 to 983
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   img_link          984 non-null    object 
 1   name              984 non-null    object 
 2   price(in Rs.)     984 non-null    int64  
 3   processor         984 non-null    object 
 4   ram               984 non-null    object 
 5   os                984 non-null    object 
 6   storage           984 non-null    object 
 7   display(in inch)  984 non-null    float64
 8   rating            688 non-null    float64
 9   no_of_ratings     688 non-null    float64
 10  no_of_reviews     688 non-null    float64
dtypes: float64(4), int64(1), object(6)
memory usage: 92.2+ KB


Данные содержат столбцы:
- тип object:
    - ссылка
    - название
    - тип процессора
    - емкость жесткого диска (storage)
    - объем оперативной памяти (ram)
    - операционная система
- числовые типы:
    - цена
    - рейтинг
    - размер экрана
    - число оценок
    - число отзывов

Можно из ram и storage выделить численные значения объемов памяти. 

### Преобразование типов данных

In [4]:
data['ram'].value_counts()

8 GB DDR4 RAM               463
16 GB DDR4 RAM              235
16 GB DDR5 RAM               72
4 GB DDR4 RAM                45
8 GB LPDDR4X RAM             23
16 GB LPDDR5 RAM             22
32 GB DDR5 RAM               21
16 GB LPDDR4X RAM            18
4 GB LPDDR4 RAM              14
16 GB Unified Memory RAM     13
8 GB DDR5 RAM                12
8 GB Unified Memory RAM       9
32 GB DDR4 RAM                7
8 GB DDR3 RAM                 7
32 GB LPDDR5 RAM              5
4 GB LPDDR4X RAM              5
16 GB LPDDR3 RAM              5
8 GB LPDDR3 RAM               4
32 GB Unified Memory RAM      1
16 GB LPDDR4 RAM              1
32 GB LPDDR4X RAM             1
16 GB DDR3 RAM                1
Name: ram, dtype: int64

Оставим в столбце ram только информацию о числе Гб

In [5]:
data['ram (GB)'] = data['ram'].apply(lambda x: int(x.split()[0]))
data = data.drop('ram', axis=1)
data['ram (GB)'].unique()

array([16,  8,  4, 32], dtype=int64)

In [6]:
data['storage'].value_counts()

512 GB SSD                                                                                                                                                                                                              575
1 TB SSD                                                                                                                                                                                                                154
256 GB SSD                                                                                                                                                                                                              123
1 TB HDD|256 GB SSD                                                                                                                                                                                                      63
1 TB HDD                                                                                                                

Из данных столбца storage создадим столбцы HDD и SSD, где укажем наличие (True) или отсутствие (False) жесткого диска и твердотельного накопителя соответственно, и столбец суммарной системной памяти в Гб.

In [7]:
def storage_GB(storage):
    gb_hdd = re.search(r'\d+ GB HDD', storage)
    tb_hdd = re.search(r'\d+ TB HDD', storage)
    gb_hdd = int(gb_hdd[0].split()[0]) if gb_hdd else 0 # количество GB жесткого диска
    tb_hdd = int(tb_hdd[0].split()[0]) if tb_hdd else 0 # количество TB жесткого диска
    gb_ssd = re.search(r'\d+ GB SSD', storage)
    tb_ssd = re.search(r'\d+ TB SSD', storage)
    gb_ssd = int(gb_ssd[0].split()[0]) if gb_ssd else 0 # количество GB твердотельного накопителя
    tb_ssd = int(tb_ssd[0].split()[0]) if tb_ssd else 0 # количество TB твердотельного накопителя
    return gb_hdd + gb_ssd + 1024*(tb_hdd + tb_ssd) if gb_hdd + gb_ssd + tb_ssd + tb_hdd>0 else None

data['HDD'] = data['storage'].apply(lambda x: 'HDD' in x)
data['SSD'] = data['storage'].apply(lambda x: 'SSD' in x)
data['storage (GB)'] = data['storage'].apply(storage_GB)
data = data.drop('storage', axis=1)

Посмотрим на число уникальных значений в колонках датасета

In [8]:
data.nunique()

img_link            584
name                506
price(in Rs.)       449
processor            59
os                   11
display(in inch)     21
rating               20
no_of_ratings       273
no_of_reviews       147
ram (GB)              4
HDD                   2
SSD                   2
storage (GB)          9
dtype: int64

In [9]:
data['os'].value_counts()

64 bit Windows 11 Operating System    527
64 bit Windows 10 Operating System    195
Windows 11 Operating System           183
Mac OS Operating System                38
Windows 10 Operating System            16
Chrome Operating System                 8
DOS Operating System                    6
64 bit Chrome Operating System          6
32 bit Windows 11 Operating System      3
64 bit DOS Operating System             1
64 bit Windows 8 Operating System       1
Name: os, dtype: int64

Среди непреобразованных категориальных переменных меньше всего типов для операционной системы. В части записей отсутствует информация о разрядной архитектуре (bit - 32 или 64), причем 32 bit присутстсвует только в 3 записях, поэтому удалим эту информацию из os. В дальнейшем будем считать будем считать предпочтительным 64 bit, поскольку этот вариант более распростроненный.

In [10]:
data['os'] = data['os'].apply(lambda x: x.strip('3264 bit'))
display(data.sample(3))

,img_link,name,price(in Rs.),processor,os,display(in inch),rating,no_of_ratings,no_of_reviews,ram (GB),HDD,SSD,storage (GB)
177,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo IdeaPad Ryzen 5 Hexa Core 5500U,47990,AMD Ryzen 5 Hexa Core Processor,Windows 11 Operating System,15.6,4.6,8.0,0.0,8,False,True,512.0
815,https://rukminim1.flixcart.com/image/312/312/l...,ASUS TUF Gaming A17 Ryzen 7 Octa Core AMD R7,85990,AMD Ryzen 7 Octa Core Processor,Windows 10 Operating System,17.3,4.5,1468.0,194.0,16,False,True,512.0
720,https://rukminim1.flixcart.com/image/312/312/x...,HP 15s Intel Core i5 12th Gen,63290,Intel Core i5 Processor (12th Gen),Windows 11 Operating System,15.6,4.1,125.0,9.0,16,False,True,512.0


### Обработка пропусков

In [11]:
# Распределение пропусков в данных в процентах
cols_null = data.isnull().mean() * 100
# колонки с пропусками
cols_with_null = cols_null[cols_null>0].sort_values(ascending=False)
display(cols_with_null)
# построим столбчатую диаграмму
fig = px.bar(cols_with_null,
             labels={'index': 'Колонки с пропусками', 'value': 'Процент пропущенных значений'},
             title='Распределение пропусков в данных в процентах',
             color_discrete_sequence=px.colors.diverging.Tealrose
);
fig.show()

rating           30.081301
no_of_ratings    30.081301
no_of_reviews    30.081301
storage (GB)      0.203252
dtype: float64

По постановке задачи, рейтинг - основополагающая информация для анализа наиболее востребованных ноутбуков. Поэтому удалим записи, где пропущен рейтинг. После этого пропушенные значения остаются только в колонке storage: заполним из медианным значением.

In [12]:
data_cleaned = data.copy()
data_cleaned = data_cleaned.dropna(subset='rating')
display(data_cleaned.isnull().mean())
data_cleaned['storage (GB)'] = data_cleaned['storage (GB)'].fillna(data_cleaned['storage (GB)'].median())
display(data_cleaned.isnull().mean().mean())

img_link            0.000000
name                0.000000
price(in Rs.)       0.000000
processor           0.000000
os                  0.000000
display(in inch)    0.000000
rating              0.000000
no_of_ratings       0.000000
no_of_reviews       0.000000
ram (GB)            0.000000
HDD                 0.000000
SSD                 0.000000
storage (GB)        0.002907
dtype: float64

0.0

### Обработка дубликатов

In [13]:
dupl_columns = list(data_cleaned.columns)

mask = data_cleaned.duplicated(subset=dupl_columns)
df_duplicates = data_cleaned[mask]
print(f'Число найденных дубликатов: {df_duplicates.shape[0]}')

data_cleaned = data_cleaned.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {data_cleaned.shape[0]}')

Число найденных дубликатов: 157
Результирующее число записей: 531


## 2.  Расчет основных статистических показателей

In [14]:
# числовые переменные
cols_numeric = ['price(in Rs.)', 'ram (GB)', 'storage (GB)',  'display(in inch)', 'rating', 'no_of_ratings', 'no_of_reviews']
data_cleaned[cols_numeric].describe(percentiles=[0.50, 0.90])

,price(in Rs.),ram (GB),storage (GB),display(in inch),rating,no_of_ratings,no_of_reviews
count,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000,531.000000
mean,70958.193974,10.538606,617.581921,15.087458,4.273446,688.165725,82.702448
std,44831.132621,4.856033,317.780651,1.577175,0.359863,1766.214946,212.978655
min,15990.000000,4.000000,128.000000,11.600000,1.600000,1.000000,0.000000
50%,59990.000000,8.000000,512.000000,15.600000,4.300000,87.000000,11.000000
90%,127000.000000,16.000000,1024.000000,15.600000,4.700000,1779.000000,226.000000
max,345390.000000,32.000000,2048.000000,35.000000,5.000000,15492.000000,2054.000000


Можно отметить большой разброс значений *цены* и *количества оценок/отзывов* (min, max, стандартное отклонение), причем максимальные значения в разы превышают соответствующие значения большинства (90%) записей. Также средние для *цены, числа оценок и отзывов, объемов оперативной и системной памяти* больше медиан, то есть распределения этих переменных  ассиметричны и являются “правосторонними” распределениями. А размер дисплея и рейтинг имеют “левосторонние” распределения, так как их медианы больше их средних.

## 3. Визуализация

Построим гистограммы и боксплоты распределений цены и рейтинга

In [15]:
labels = {'price(in Rs.)': 'цена', 'rating': 'рейтинг'}

for col in ['price(in Rs.)', 'rating']:
    hist = px.histogram(
        data_cleaned,
        x=col,
        color_discrete_sequence=px.colors.diverging.Tealrose,
        title=f'Распределение переменной {labels[col]}'
    )
    hist.show()
    hist.write_html(f"plotly/hist_{labels[col]}.html")
    box = px.box(
        data_cleaned,
        x=col,
        orientation='h',
        color_discrete_sequence=px.colors.diverging.Tealrose,
        title=f'Распределение переменной {labels[col]}'
    )
    box.show()
    box.write_html(f"plotly/box_{labels[col]}.html")

Как уже было отмечено, цены распеределены ассиметрично - "хвост" справа, а у распределения рейтинга "хвост" слева. То есть присутствует мало ноутбуков с ценой выше 136k Rs, и с рейтингом меньше 3.5

Теперь построим совместное распределение этих признаков и проверим, есть ли между ними взаимосвязь.

In [16]:
scatter = px.scatter(
    data_frame=data_cleaned,
    y='rating',
    x='price(in Rs.)',
    color_discrete_sequence=px.colors.diverging.Tealrose,
    title='Зависимость рейтинга от цены'
)
scatter.show()
scatter.write_html("plotly/scatter_rating_price.html")

Можно заметить, что ноутбуки стоимостью от 200k Rs имеют рейтинг не ниже 4. Однако самые высокие и самые низкие оценки не относятся к самой высокой и самой низкой ценам, и несколько ноутбуков с одной и той же ценой могут иметь различные значения рейтнга. Иначе говоря, есть некоторая положилетьная корреляция рейтинга и цены ноутбука, но несильная. Возможно, пользователи, выбирая более дорогой вариант, выдвигают больше требований к качеству и запрашивают определенные технические характеристики, и из-за этого низкие оценки встречаются при ценах около средней стоимости (~70k) и выше.

Оценим влияние операционной системы на рейтинг. Перед этим посмотрим на распределение признака операционная система.

In [17]:
# гистограмма распределения
hist = px.histogram(
    data_cleaned,
    x='os',
    color_discrete_sequence=px.colors.diverging.Tealrose,
    title=f'Распределение переменной операционная система'
)
hist.show()
hist.write_html("plotly/os.html")

# коробчатая диаграмма
fig = px.box(
    data_frame=data_cleaned, 
    x='rating',
    color='os',
    orientation='h',
    color_discrete_sequence=px.colors.diverging.Tealrose[::2],
    title='Распределение рейтинга в зависимости от операционной системы'
)
fig.show()
fig.write_html("plotly/box_rating_os.html")

# группируем данные по операционным системам
bar_data = (data_cleaned
            .groupby(
    by='os',
    as_index=False
)[['rating']].median())
# столбчатая диаграмма
fig = px.bar(
    data_frame=bar_data, 
    x='os', 
    y='rating',
    color='os', 
    orientation='v',
    color_discrete_sequence=px.colors.diverging.Tealrose[::2],
    title='Зависимость медианного рейтинга от операционной системы'
)
fig.show()
fig.write_html("plotly/bar_rating_os.html")


Наибольшее медианное значение рейтинга наблюдается для MAC OS (4.7), за ним следуют показатели для Windows 10 и 11 (больше 4), при этом у Windows самые большие разбросы оценок (есть случаи рейтинга меньше 3.5, есть и значения 5).

Наиболее популярной и востребованной ОС, судя по графикам, следует считать **MAC**. Хотя не стоит забывать, что данных о ноутбуках с MAC OS в датасете немного. Если выбирать между 10 и 11 Windows: выпускается больше ноутбуков с **Windows 11**, и для них нет оченок меньше либо равных 3.0.

Проанализируем, какие торговые марки более популярны, и влияет ли бренд на рейтинг

In [18]:
# выделем столбец с брендом (первое слово из name)
data_cleaned['brand_name'] = data_cleaned['name'].apply(lambda x: x.lower().split()[0])

# гистограмма распределения
hist = px.histogram(
    data_cleaned,
    x='brand_name',
    color_discrete_sequence=px.colors.diverging.Tealrose,
    title=f'Распределение переменной бренд'
)
hist.show()
hist.write_html("plotly/brand_name.html")

brands = data_cleaned['brand_name'].value_counts()

In [19]:
# выберем 5 наиболее часто встречающихся марки (больше 40 раз)
data_filt = pd.DataFrame()
for name in brands.head(5).index:
    data_filt = pd.concat([data_filt, data_cleaned[data_cleaned['brand_name']==name]])

# коробчатая диаграмма
fig = px.box(
    data_frame=data_filt, 
    x='rating',
    color='brand_name',
    orientation='h',
    color_discrete_sequence=px.colors.diverging.Tealrose[::2],
    title='Распределение рейтинга в зависимости от бренда'
)
fig.show()
fig.write_html("plotly/rating_brand_name.html")

Среди пяти самых распространенных тогровых марок наибольший медианный рейтинг наблюдается у **ASUS**, так же как и наибольшее количество продаваемы ноутбуков, то есть ноутбуки этого бренда более популярны, чем остальных

Построим многоуровневую столбчатую диаграмму, которая показывает зависимость медианного рейтинга от типа хранения системной памяти - признаков HDD и SSD

In [20]:
bar_data = (data_cleaned.groupby(
    by=['HDD', 'SSD'],
    as_index=False
)[['rating']].median())

fig = px.bar(data_frame=bar_data, 
             y='rating', 
             x='SSD',
             color='HDD',
             height=500, 
             width=1000,
             color_discrete_sequence=px.colors.diverging.Tealrose[::5],
             title='Зависимость медианного рейтинга от типа хранения системной памяти - признаков HHD и SDD')
fig.show()
fig.write_html("plotly/rating_SSD_HDD.html")

При наличии твердотельного накопителя SSD медиана рейтинга равна 4.3 и не зависит от наличия жесткого диска. При наличии только жесткого диска без твердотельного накопителя рейтинг ниже - 4. Следовательно, **наличие SSD** будет преимуществом.

Есть ли взамисвязь рейтинга с объемом памяти? Оценим наиболее подходящий объем системной памяти.

In [21]:
# гистограмма распределения storage
hist = px.histogram(
    data_cleaned,
    x='storage (GB)',
    color_discrete_sequence=px.colors.diverging.Tealrose,
    title=f'Распределение переменной объем системной памяти'
)
hist.show()
hist.write_html("plotly/storage.html")

# взамисвязь рейтинга с объемом памяти
box = px.box(
    data_frame=data_cleaned,
    x='rating',
    y = 'storage (GB)',
    color='storage (GB)',
    orientation='h',
    color_discrete_sequence=px.colors.diverging.Tealrose[::2],
    title='Зависимость рейтинга от объема системной памяти'
)
box.show()
box.write_html("plotly/box_rating_storage.html")

По медианному значению рейтинга выделяется объем 1536 (1 Тб + 512 Гб), однако максимальный рейтинг в этом случае всего лишь 4.5 - там слишком маленькая выборка. Более целесообразно обратить внимание на те варианты, где присутствуют оценки 5.0 (они и более распространенные): 
- 512 Гб, 
- 256 Гб, 
- 1024 Гб (1 Тб).

Для 512 Гб и 1 Тб медиана рейтинга составляет 4.3, что равно медиане оценкок по всему датасету. Эти два значения storage и следует считать оптимальными, между ними пользователи могут выбрать тот объем, что необходим для их целей. Отметим, что все же намного чаще встречается объем **512 Гб**.

Оценим, как влияет на рейтинг объем оперативной памяти (ram)

In [22]:
# гистограмма распределения ram
hist = px.histogram(
    data_cleaned,
    x='ram (GB)',
    color_discrete_sequence=px.colors.diverging.Tealrose,
    title=f'Распределение переменной объем оперативной памяти'
)
hist.show()
hist.write_html("plotly/ram.html")

# взамисвязь рейтинга с объемом памяти
box = px.box(
    data_frame=data_cleaned,
    x='rating',
    y='ram (GB)',
    color='ram (GB)',
    orientation='h',
    color_discrete_sequence=px.colors.diverging.Tealrose[::2],
    title='Зависимость рейтинга от объема оперативной памяти'
)
box.show()
box.write_html("plotly/box_rating_ram.html")

Между рейтингом и объемом оперативной пямяти слабая положительная связь. Наибольшая медиана наблюдается для оперативной пямяти 32 Гб, однако в данных только 7 таких экземляров, поэтому надежнее опираться на более многочисленную выборку и вторую по медианному значению рейтинга - **16 Гб**.

Попробуем выявить связь рейтинга и размера экрана.

In [23]:
# гистограмма распределения ram
hist = px.histogram(
    data_cleaned,
    x='display(in inch)',
    color_discrete_sequence=px.colors.diverging.Tealrose,
    title=f'Распределение переменной размер экрана'
)
hist.show()

displays = data_cleaned['display(in inch)'].value_counts()
displays.head()
hist.write_html("plotly/display.html")

Так как распределение рамеров крайне неравномерное, то будет малоинформативно брать в расчет редкие значения. Рассмотрим те, что встречаются в датасете больше 10 раз

In [24]:
# возьмем 3 наиболее распространенных размера
box_data = pd.DataFrame()
for dis in displays.head(3).index:
    box_data = pd.concat([box_data, data_cleaned[data_cleaned['display(in inch)']==dis]])

box = px.box(
    data_frame=box_data,
    x='rating',
    y='display(in inch)',
    color='display(in inch)',
    orientation='h',
    color_discrete_sequence=px.colors.diverging.Tealrose[::2],
    title='Зависимость рейтинга от размера экрана'
)
box.show()
box.write_html("plotly/box_rating_display.html")

Наиболее распространены размеры 15.6, 14 и 13.3 дюймов. Для большинства ноутбуков с экраном 14 дюймов значения рейтинга лежат в интервале от 3.7 до 4.8, то есть 5.0 здесь скорее выброс, а медианное значение рейтинга для этой группы меньше, чем для **15.6 и 13.3**. Эти два размера будем считать оптимальными. **15.6** - самый популярный, и медиана рейтинга 4.3, что равно медиане оценкок по всему датасету. А **13.3** может быть в приоритете из-за меньшего размера и веса для пользователей, кто часто носит ноутбук с собой, это ткаже стоит учитывать, к тому же медиана рейтинга для данного размера больше - 4.5

## 4. Матрица корреляции и heatmap для нее

In [25]:
corr = data_cleaned.corr(numeric_only=True)
corr

,price(in Rs.),display(in inch),rating,no_of_ratings,no_of_reviews,ram (GB),HDD,SSD,storage (GB)
price(in Rs.),1.000000,0.078550,0.321031,-0.121662,-0.126380,0.701763,-0.110181,0.133024,0.349205
display(in inch),0.078550,1.000000,0.013906,-0.084972,-0.078826,0.083590,0.099757,0.005451,0.159986
rating,0.321031,0.013906,1.000000,0.074955,0.062850,0.293458,-0.232080,0.277323,-0.051254
no_of_ratings,-0.121662,-0.084972,0.074955,1.000000,0.978092,-0.172299,0.043333,-0.086237,-0.077491
no_of_reviews,-0.126380,-0.078826,0.062850,0.978092,1.000000,-0.169990,0.066448,-0.099467,-0.057660
ram (GB),0.701763,0.083590,0.293458,-0.172299,-0.169990,1.000000,-0.167978,0.187269,0.249168
HDD,-0.110181,0.099757,-0.232080,0.043333,0.066448,-0.167978,1.000000,-0.655088,0.631692
SSD,0.133024,0.005451,0.277323,-0.086237,-0.099467,0.187269,-0.655088,1.000000,-0.314365
storage (GB),0.349205,0.159986,-0.051254,-0.077491,-0.057660,0.249168,0.631692,-0.314365,1.000000


In [26]:
# матрица для признаков, исключая рейтинг
fig = px.imshow(
    corr.drop('rating').drop('rating', axis=1),
    color_continuous_scale=px.colors.diverging.Tealrose,
    title='Матрица корреляции (исключая рейтинг) - тепловая карта')
fig.show()
fig.write_html("plotly/corr.html")

# корреляции с рейтингом
rating_corr = corr['rating'].sort_values(ascending=False)[1:]
bar = px.bar(rating_corr,
             color=rating_corr.index,
             color_discrete_sequence=px.colors.diverging.Tealrose,
             title='Диаграмма корреляций рейтинга с остальными переменными')
bar.show()
bar.write_html("plotly/rating_corr.html")

Сильная корреляция наблюдается между количеством отзывов и оценок, что логично: когда пишут отзыв обычно ставят и оценку. Также есть средняя связь объема системной памяти и типом ее носителя (чаще встречаются твердотельные накопители SSD; реже бывает оба типа сразу, но это увеличивает общий объем). Обнаруживается корреляция 0.7 между цены с оперативной памятью и слабая корреляция (0.35) с системной памятью, то есть можно предположить, что цена помышается при большей доступной памяти.

Переменная рейтинга слабо (~0.3) коррелирует с ценой и с оперативной памятью, и еще более слабо - с SSD (положительно) и HDD (отрицательно), что подтверждает ранее сделанные выводы.

## 5. Общий вывод по исследуемому датасету

В результате анализа выделены оптимальные параметры для ноутбука:
- операционная система - MAC OS или Windows 11 (последняя более рспространена)
- тип хранения системной памяти - твердотельный накопитель (SSD)
- объем системной памяти - 1 Тб и 512 Гб (последний более рспространен)
- объем оперативной памяти - 16 Гб
- размер экрана - 13.3 и 15.6 дюймов (последний более рспространен)

Возможные способы применения полученных результатов анализа данных:
 1. рекомендация производителям ноутбуков выбирать популярные параметры: системная память - SSD 512 Гб, экран - 15.6 дюймов, оперативная память - 16 Гб, операционная система - Windows 11 (или выпускать компьютер без ОС, с возможностью установки желаемой системы при покупке). При этом следует учитывать и другие варианты, например, большие объемы памяти или меньший размер дисплея, которые могут быть актуальны для отдельных групп покупателей.
 
 2. для покупателей данные о рейтинге в зависимости от различных параметров могут помочь при выборе нового ноутбука. Основываясь на приведенных выше характеристиках, отобранных по медианным значениям рейтинга и по частоте встречаемости, пользователь может выбрать оптимальные параметры, необходимые для его задач, а также учесть распределение торговых марок (среди рассмотренных в датасете приоритетным брендом является ASUS). Для конкретизации выбора уже, конечно, потребуется прочитать отзывы на выделенные модели ноутбуков.